In [23]:
import pandas as pd
import numpy as np 
import re
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
from pathlib import Path
from shapely.ops import transform
from pyproj import Transformer

In [24]:
df_risk_data = pd.read_csv('../data/bdaic created tables/df_high_risk.csv', dtype={'PERSON ID': str}) 

In [25]:
df_risk_data.head()

,PERSON ID,CURRENT AGE,COMMITMENT COUNTY,RESPONSIBLE COUNTY,GENDER,REMOVAL ZIP CODE,PLACEMENT ZIP CODE,LOCATION BEGIN DATE,LOCATION END DATE,Decision-Making (Judgement),...,Oppositional(Non-compliance with Authority),Relationship Permanence,Social Functioning,Independent Living Skills,Living Situation,Physical Abuse,Runaway*,Youth Residential Stability,County,Region
0,353258,17,Madison,Madison,MALE,37040.0,37040.0,2023-06-13,2024-05-25,0,...,0,0,0,0.0,0,0,0,0.0,Madison,West
1,706512,17,Sullivan,Sullivan,FEMALE,37642.0,37642.0,2024-11-18,NaN,0,...,0,0,0,0.0,0,0,0,0.0,Sullivan,East
2,1348374,17,Warren,Warren,FEMALE,37110.0,37110.0,2023-06-02,2023-09-30,0,...,0,0,0,0.0,0,0,0,0.0,Warren,Middle
3,2470086,18,Hamilton,Hamilton,MALE,37218.0,37218.0,2024-09-17,NaN,1,...,0,1,1,1.0,0,0,0,1.0,Hamilton,East
4,3965628,21,Davidson,Davidson,MALE,37725.0,37725.0,2021-12-22,2022-08-31,1,...,1,0,0,0.0,0,0,0,0.0,Davidson,Middle


In [26]:
df_risk_data.columns.tolist()

['PERSON ID',
 'CURRENT AGE',
 'COMMITMENT COUNTY',
 'RESPONSIBLE COUNTY',
 'GENDER',
 'REMOVAL ZIP CODE',
 'PLACEMENT ZIP CODE',
 'LOCATION BEGIN DATE',
 'LOCATION END DATE',
 'Decision-Making (Judgement)',
 'Impulsivity/Hyperactivity',
 'School Behavior',
 'Substance Use',
 'Spiritual/Religious',
 'Traditions And Rituals',
 'Youth Involvement With Care',
 'Anger Control',
 'Anxiety',
 'Depression',
 'Emotional And/Or Physical Dysregulation',
 'Numbing',
 'Medical/Physical',
 'Psychosis (Thought Disorder)',
 'Danger To Others',
 'Intentional Misbehavior',
 'Non-Suicidal Self-Injurious Behavior',
 'Other Self-Harm (Recklessness)',
 'Suicide Risk',
 'Sexual Aggression',
 'Sexual Development',
 'Adjustment To Trauma',
 'Cultural Stress',
 'Emotional Abuse',
 'Environmental Influences',
 'Medical Trauma',
 'Sexual Abuse',
 'Attachment Difficulties',
 'Community Life',
 'Family Functioning',
 'Family Strengths',
 'Interpersonal/Social Connectedness',
 'Natural Supports',
 'Neglect',
 'Oppo

In [27]:
calc_cols = ['Decision-Making (Judgement)',
 'Impulsivity/Hyperactivity',
 'School Behavior',
 'Substance Use',
 'Spiritual/Religious',
 'Traditions And Rituals',
 'Youth Involvement With Care',
 'Anger Control',
 'Anxiety',
 'Depression',
 'Emotional And/Or Physical Dysregulation',
 'Numbing',
 'Medical/Physical',
 'Psychosis (Thought Disorder)',
 'Danger To Others',
 'Intentional Misbehavior',
 'Non-Suicidal Self-Injurious Behavior',
 'Other Self-Harm (Recklessness)',
 'Suicide Risk',
 'Sexual Aggression',
 'Sexual Development',
 'Adjustment To Trauma',
 'Cultural Stress',
 'Emotional Abuse',
 'Environmental Influences',
 'Medical Trauma',
 'Sexual Abuse',
 'Attachment Difficulties',
 'Community Life',
 'Family Functioning',
 'Family Strengths',
 'Interpersonal/Social Connectedness',
 'Natural Supports',
 'Neglect',
 'Oppositional(Non-compliance with Authority)',
 'Relationship Permanence',
 'Social Functioning',
 'Independent Living Skills',
 'Living Situation',
 'Physical Abuse',
 'Runaway*',
 'Youth Residential Stability']

In [28]:
# Mental Health categories
mh_behavioral_impulse_control = [
    'Decision-Making (Judgement)',
    'Impulsivity/Hyperactivity', 
    'School Behavior',
    'Substance Use'
]

mh_cultural_spiritual_engagement = [
    'Spiritual/Religious',
    'Traditions And Rituals',
    'Youth Involvement With Care'
]

mh_mood_emotional_regulation = [
    'Anger Control',
    'Anxiety', 
    'Depression',
    'Emotional And/Or Physical Dysregulation',
    'Numbing'
]

mh_physical_medical = [
    'Medical/Physical'
]

mh_psychotic_thought = [
    'Psychosis (Thought Disorder)'
]

mh_risk_safety = [
    'Danger To Others',
    'Intentional Misbehavior',
    'Non-Suicidal Self-Injurious Behavior', 
    'Other Self-Harm (Recklessness)',
    'Suicide Risk'
]

mh_sexual_developmental = [
    'Sexual Aggression',
    'Sexual Development'
]

mh_trauma_stress = [
    'Adjustment To Trauma',
    'Cultural Stress',
    'Emotional Abuse',
    'Environmental Influences',
    'Medical Trauma', 
    'Sexual Abuse'
]

# Group Mental Health categories
mental_health_categories = {
    'Behavioral & Impulse Control': mh_behavioral_impulse_control,
    'Cultural, Spiritual, and Engagement Factors': mh_cultural_spiritual_engagement,
    'Mood & Emotional Regulation': mh_mood_emotional_regulation,
    'Physical & Medical': mh_physical_medical,
    'Psychotic & Thought Disturbances': mh_psychotic_thought,
    'Risk & Safety': mh_risk_safety,
    'Sexual & Developmental Concerns': mh_sexual_developmental,
    'Trauma & Stress-Related': mh_trauma_stress
}

# Supportive Adult Relationship categories
sar_family_relationships = [
    'Family Functioning', 
    'Family Strengths',
    'Relationship Permanence',
] 

sar_social_relationships = [
    'Community Life', 
    'Interpersonal/Social Connectedness', 
    'Natural Supports', 
    'Social Functioning'
]

sar_barriers_to_support = [
    'Attachment Difficulties',
    'Oppositional(Non-compliance with Authority)',
    'Neglect'
]

# Group Supportive Adult Relationship categories
supportive_adult_relationships = {
    'Family Relationships': sar_family_relationships,
    'Social Relationships': sar_social_relationships,
    'Barriers to Support': sar_barriers_to_support
}

# Housing Instability categories
hi_skills_readiness = [
    'Independent Living Skills',
]

hi_current_living_situation = [
    'Living Situation',
    'Youth Residential Stability' 
]

hi_barriers_risk = [
    'Physical Abuse', 
    'Runaway*'
]

# Housing Instability categories
housing_instability = {
    'Skills & Readiness': hi_skills_readiness,
    'Current Living Situation': hi_current_living_situation,
    'Barriers & Risks': hi_barriers_risk
}

# Group all categories
assessment_categories = {
    'Mental Health': mental_health_categories,
    'Supportive Adult Relationships': supportive_adult_relationships,
    'Housing Instability': housing_instability
}

In [73]:
df_risk_data_bucket = pd.read_csv('../data/bdaic created tables/df_high_risk_buckets.csv', dtype={'PERSON ID': str}) 

In [74]:
df_risk_data_buckets.head()

,PERSON ID,CURRENT AGE,COMMITMENT COUNTY,RESPONSIBLE COUNTY,GENDER,REMOVAL ZIP CODE,PLACEMENT ZIP CODE,LOCATION BEGIN DATE,LOCATION END DATE,Mental_Health_Behavioral_Impulse_Control,...,Mental_Health_Psychotic_Thought_Disturbances,Mental_Health_Risk_Safety,Mental_Health_Sexual_Developmental_Concerns,Mental_Health_Trauma_Stress_Related,Supportive_Adult_Relationships_Family_Relationships,Supportive_Adult_Relationships_Social_Relationships,Supportive_Adult_Relationships_Barriers_to_Support,Housing_Instability_Skills_Readiness,Housing_Instability_Current_Living_Situation,Housing_Instability_Barriers_Risks
0,353258,17,Madison,Madison,MALE,37040.0,37040.0,2023-06-13,2024-05-25,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0
1,706512,17,Sullivan,Sullivan,FEMALE,37642.0,37642.0,2024-11-18,NaN,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0
2,1348374,17,Warren,Warren,FEMALE,37110.0,37110.0,2023-06-02,2023-09-30,0.0,...,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0
3,2470086,18,Hamilton,Hamilton,MALE,37218.0,37218.0,2024-09-17,NaN,NaN,...,0.0,NaN,0.0,0.0,NaN,NaN,0.0,NaN,NaN,0.0
4,3965628,21,Davidson,Davidson,MALE,37725.0,37725.0,2021-12-22,2022-08-31,NaN,...,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0


In [33]:
# Percent of ones per county for each flag column
pct_ones_by_county = (
    merged.groupby('County')[calc_cols]
    .mean()
    .mul(100)
    .round(2)
    .reset_index()
)
pct_ones_by_county

,County,Decision-Making (Judgement),Impulsivity/Hyperactivity,School Behavior,Substance Use,Spiritual/Religious,Traditions And Rituals,Youth Involvement With Care,Anger Control,Anxiety,...,Natural Supports,Neglect,Oppositional(Non-compliance with Authority),Relationship Permanence,Social Functioning,Independent Living Skills,Living Situation,Physical Abuse,Runaway*,Youth Residential Stability
0,Anderson,40.37,22.02,11.93,19.27,77.98,0.92,19.10,26.61,41.28,...,79.82,59.63,30.28,49.54,37.61,18.28,17.43,22.94,15.60,8.89
1,Bedford,50.00,32.00,12.00,18.00,60.00,0.00,11.43,22.00,22.00,...,78.00,34.00,26.00,44.00,26.00,14.29,26.00,16.00,12.00,14.29
2,Benton,76.92,61.54,0.00,15.38,61.54,0.00,9.09,69.23,61.54,...,84.62,46.15,46.15,69.23,61.54,0.00,76.92,38.46,0.00,18.18
3,Bledsoe,26.67,20.00,6.67,26.67,53.33,0.00,0.00,20.00,26.67,...,66.67,20.00,13.33,40.00,33.33,0.00,26.67,13.33,0.00,0.00
4,Blount,64.44,36.67,7.78,12.22,75.56,1.11,9.86,20.00,48.89,...,75.56,67.78,21.11,83.33,22.22,15.49,23.33,38.89,6.67,18.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,Wayne,54.84,38.71,16.13,25.81,48.39,0.00,20.00,25.81,32.26,...,74.19,70.97,16.13,83.87,51.61,10.00,29.03,9.68,6.45,13.33
91,Weakley,70.59,29.41,11.76,35.29,70.59,0.00,18.75,35.29,41.18,...,88.24,52.94,47.06,88.24,47.06,13.33,52.94,23.53,11.76,6.25
92,White,52.63,39.47,7.89,28.95,68.42,0.00,15.15,23.68,36.84,...,86.84,73.68,13.16,39.47,28.95,12.12,26.32,23.68,10.53,15.62
93,Williamson,61.54,33.85,20.00,23.08,67.69,0.00,13.95,27.69,23.08,...,80.00,32.31,40.00,47.69,41.54,13.95,29.23,12.31,21.54,13.95


In [34]:
pct_ones_by_county.to_csv('pct_ones_risk_by_county.csv', index=False)

In [76]:
df_all_risk = pd.merge(
    df_risk_data,
    df_risk_data_buckets,
    on="PERSON ID",       # common column
    how="left"           # keep all rows from df_extreme_risk_data
)

In [77]:
df_all_risk.head()

,PERSON ID,CURRENT AGE_x,COMMITMENT COUNTY_x,RESPONSIBLE COUNTY_x,GENDER_x,REMOVAL ZIP CODE_x,PLACEMENT ZIP CODE_x,LOCATION BEGIN DATE_x,LOCATION END DATE_x,Decision-Making (Judgement),...,Mental_Health_Psychotic_Thought_Disturbances,Mental_Health_Risk_Safety,Mental_Health_Sexual_Developmental_Concerns,Mental_Health_Trauma_Stress_Related,Supportive_Adult_Relationships_Family_Relationships,Supportive_Adult_Relationships_Social_Relationships,Supportive_Adult_Relationships_Barriers_to_Support,Housing_Instability_Skills_Readiness,Housing_Instability_Current_Living_Situation,Housing_Instability_Barriers_Risks
0,353258,17,Madison,Madison,MALE,37040.0,37040.0,2023-06-13,2024-05-25,0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0
1,706512,17,Sullivan,Sullivan,FEMALE,37642.0,37642.0,2024-11-18,NaN,0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0
2,1348374,17,Warren,Warren,FEMALE,37110.0,37110.0,2023-06-02,2023-09-30,0,...,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0
3,2470086,18,Hamilton,Hamilton,MALE,37218.0,37218.0,2024-09-17,NaN,1,...,0.0,NaN,0.0,0.0,NaN,NaN,0.0,NaN,NaN,0.0
4,3965628,21,Davidson,Davidson,MALE,37725.0,37725.0,2021-12-22,2022-08-31,1,...,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0


In [79]:
df_all_risk.columns.tolist()

['PERSON ID',
 'CURRENT AGE_x',
 'COMMITMENT COUNTY_x',
 'RESPONSIBLE COUNTY_x',
 'GENDER_x',
 'REMOVAL ZIP CODE_x',
 'PLACEMENT ZIP CODE_x',
 'LOCATION BEGIN DATE_x',
 'LOCATION END DATE_x',
 'Decision-Making (Judgement)',
 'Impulsivity/Hyperactivity',
 'School Behavior',
 'Substance Use',
 'Spiritual/Religious',
 'Traditions And Rituals',
 'Youth Involvement With Care',
 'Anger Control',
 'Anxiety',
 'Depression',
 'Emotional And/Or Physical Dysregulation',
 'Numbing',
 'Medical/Physical',
 'Psychosis (Thought Disorder)',
 'Danger To Others',
 'Intentional Misbehavior',
 'Non-Suicidal Self-Injurious Behavior',
 'Other Self-Harm (Recklessness)',
 'Suicide Risk',
 'Sexual Aggression',
 'Sexual Development',
 'Adjustment To Trauma',
 'Cultural Stress',
 'Emotional Abuse',
 'Environmental Influences',
 'Medical Trauma',
 'Sexual Abuse',
 'Attachment Difficulties',
 'Community Life',
 'Family Functioning',
 'Family Strengths',
 'Interpersonal/Social Connectedness',
 'Natural Supports',
 '

In [80]:
df_all_risk.drop(columns=['CURRENT AGE_x',
 'COMMITMENT COUNTY_x',
 'RESPONSIBLE COUNTY_x',
 'GENDER_x',
 'REMOVAL ZIP CODE_x',
 'PLACEMENT ZIP CODE_x',
 'LOCATION BEGIN DATE_x',
 'LOCATION END DATE_x'])


,PERSON ID,Decision-Making (Judgement),Impulsivity/Hyperactivity,School Behavior,Substance Use,Spiritual/Religious,Traditions And Rituals,Youth Involvement With Care,Anger Control,Anxiety,...,Mental_Health_Psychotic_Thought_Disturbances,Mental_Health_Risk_Safety,Mental_Health_Sexual_Developmental_Concerns,Mental_Health_Trauma_Stress_Related,Supportive_Adult_Relationships_Family_Relationships,Supportive_Adult_Relationships_Social_Relationships,Supportive_Adult_Relationships_Barriers_to_Support,Housing_Instability_Skills_Readiness,Housing_Instability_Current_Living_Situation,Housing_Instability_Barriers_Risks
0,353258,0,0,0,0,0,0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0
1,706512,0,0,0,0,0,0,0.0,0,0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0
2,1348374,0,0,0,0,0,0,0.0,0,0,...,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0
3,2470086,1,0,0,1,1,0,0.0,1,1,...,0.0,NaN,0.0,0.0,NaN,NaN,0.0,NaN,NaN,0.0
4,3965628,1,1,0,1,0,0,0.0,0,0,...,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5662,135541144,0,0,0,0,1,0,0.0,0,0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0
5663,135654245,1,0,0,0,1,0,1.0,0,0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
5664,135694181,0,0,0,0,1,0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0
5665,135731227,0,1,0,0,1,0,0.0,0,1,...,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0


In [81]:
 # Rename columns to remove '_x' suffix
df_all_risk = df_all_risk.rename(
    columns={col: col.replace('_y', '') for col in df_all_extreme.columns if col.endswith('_y')}
)

In [83]:
# Percent of ones per county for each flag column
pct_ones_by_county_risk = (
    df_all_risk.groupby('County')[calc_cols2]
    .mean()
    .mul(100)
    .round(2)
    .reset_index()
)
pct_ones_by_county_risk

,County,Decision-Making (Judgement),Impulsivity/Hyperactivity,School Behavior,Substance Use,Spiritual/Religious,Traditions And Rituals,Youth Involvement With Care,Anger Control,Anxiety,...,Mental_Health_Psychotic_Thought_Disturbances,Mental_Health_Risk_Safety,Mental_Health_Sexual_Developmental_Concerns,Mental_Health_Trauma_Stress_Related,Supportive_Adult_Relationships_Family_Relationships,Supportive_Adult_Relationships_Social_Relationships,Supportive_Adult_Relationships_Barriers_to_Support,Housing_Instability_Skills_Readiness,Housing_Instability_Current_Living_Situation,Housing_Instability_Barriers_Risks
0,Anderson,40.37,22.02,11.93,19.27,77.98,0.92,19.10,26.61,41.28,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Bedford,50.00,32.00,12.00,18.00,60.00,0.00,11.43,22.00,22.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Benton,76.92,61.54,0.00,15.38,61.54,0.00,9.09,69.23,61.54,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bledsoe,26.67,20.00,6.67,26.67,53.33,0.00,0.00,20.00,26.67,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Blount,64.44,36.67,7.78,12.22,75.56,1.11,9.86,20.00,48.89,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,Wayne,54.84,38.71,16.13,25.81,48.39,0.00,20.00,25.81,32.26,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91,Weakley,70.59,29.41,11.76,35.29,70.59,0.00,18.75,35.29,41.18,...,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0
92,White,52.63,39.47,7.89,28.95,68.42,0.00,15.15,23.68,36.84,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93,Williamson,61.54,33.85,20.00,23.08,67.69,0.00,13.95,27.69,23.08,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [84]:
pct_ones_by_county_risk.to_csv('risk_by_county_mapping.csv', index=False)

## Exreme

In [35]:
df_extreme_risk_data = pd.read_csv('../data/bdaic created tables/df_extreme_risk.csv', dtype={'PERSON ID': str}) 

In [36]:
df_extreme_risk_data.head()

,PERSON ID,CURRENT AGE,COMMITMENT COUNTY,RESPONSIBLE COUNTY,GENDER,REMOVAL ZIP CODE,PLACEMENT ZIP CODE,LOCATION BEGIN DATE,LOCATION END DATE,Decision-Making (Judgement),...,Oppositional(Non-compliance with Authority),Relationship Permanence,Social Functioning,Independent Living Skills,Living Situation,Physical Abuse,Runaway*,Youth Residential Stability,County,Region
0,353258,17,Madison,Madison,MALE,37040.0,37040.0,2023-06-13,2024-05-25,0,...,0,0,0,0.0,0,0,0,0.0,Madison,West
1,706512,17,Sullivan,Sullivan,FEMALE,37642.0,37642.0,2024-11-18,NaN,0,...,0,0,0,0.0,0,0,0,0.0,Sullivan,East
2,1348374,17,Warren,Warren,FEMALE,37110.0,37110.0,2023-06-02,2023-09-30,0,...,0,0,0,0.0,0,0,0,0.0,Warren,Middle
3,2470086,18,Hamilton,Hamilton,MALE,37218.0,37218.0,2024-09-17,NaN,0,...,0,0,0,0.0,0,0,0,0.0,Hamilton,East
4,3965628,21,Davidson,Davidson,MALE,37725.0,37725.0,2021-12-22,2022-08-31,0,...,1,0,0,0.0,0,0,0,0.0,Davidson,Middle


In [58]:
df_extreme_risk_data.head()

,PERSON ID,CURRENT AGE,COMMITMENT COUNTY,RESPONSIBLE COUNTY,GENDER,REMOVAL ZIP CODE,PLACEMENT ZIP CODE,LOCATION BEGIN DATE,LOCATION END DATE,Decision-Making (Judgement),...,Oppositional(Non-compliance with Authority),Relationship Permanence,Social Functioning,Independent Living Skills,Living Situation,Physical Abuse,Runaway*,Youth Residential Stability,County,Region
0,353258,17,Madison,Madison,MALE,37040.0,37040.0,2023-06-13,2024-05-25,0,...,0,0,0,0.0,0,0,0,0.0,Madison,West
1,706512,17,Sullivan,Sullivan,FEMALE,37642.0,37642.0,2024-11-18,NaN,0,...,0,0,0,0.0,0,0,0,0.0,Sullivan,East
2,1348374,17,Warren,Warren,FEMALE,37110.0,37110.0,2023-06-02,2023-09-30,0,...,0,0,0,0.0,0,0,0,0.0,Warren,Middle
3,2470086,18,Hamilton,Hamilton,MALE,37218.0,37218.0,2024-09-17,NaN,0,...,0,0,0,0.0,0,0,0,0.0,Hamilton,East
4,3965628,21,Davidson,Davidson,MALE,37725.0,37725.0,2021-12-22,2022-08-31,0,...,1,0,0,0.0,0,0,0,0.0,Davidson,Middle


In [55]:
df_extreme_risk_data_bucket = pd.read_csv('../data/bdaic created tables/df_extreme_risk_buckets.csv', dtype={'PERSON ID': str}) 

In [56]:
df_extreme_risk_data_bucket.head()

,PERSON ID,CURRENT AGE,COMMITMENT COUNTY,RESPONSIBLE COUNTY,GENDER,REMOVAL ZIP CODE,PLACEMENT ZIP CODE,LOCATION BEGIN DATE,LOCATION END DATE,Mental_Health_Behavioral_Impulse_Control,...,Mental_Health_Sexual_Developmental_Concerns,Mental_Health_Trauma_Stress_Related,Supportive_Adult_Relationships_Family_Relationships,Supportive_Adult_Relationships_Social_Relationships,Supportive_Adult_Relationships_Barriers_to_Support,Housing_Instability_Skills_Readiness,Housing_Instability_Current_Living_Situation,Housing_Instability_Barriers_Risks,County,Region
0,353258,17,Madison,Madison,MALE,37040.0,37040.0,2023-06-13,2024-05-25,NaN,...,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,Madison,West
1,706512,17,Sullivan,Sullivan,FEMALE,37642.0,37642.0,2024-11-18,NaN,NaN,...,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,Sullivan,East
2,1348374,17,Warren,Warren,FEMALE,37110.0,37110.0,2023-06-02,2023-09-30,NaN,...,0.0,NaN,NaN,1.0,NaN,NaN,0.0,0.0,Warren,Middle
3,2470086,18,Hamilton,Hamilton,MALE,37218.0,37218.0,2024-09-17,NaN,NaN,...,0.0,0.0,NaN,1.0,NaN,NaN,NaN,NaN,Hamilton,East
4,3965628,21,Davidson,Davidson,MALE,37725.0,37725.0,2021-12-22,2022-08-31,NaN,...,0.0,NaN,0.0,NaN,1.0,0.0,0.0,0.0,Davidson,Middle


In [62]:
df_all_extreme = pd.merge(
    df_extreme_risk_data,
    df_extreme_risk_data_bucket,
    on="PERSON ID",       # common column
    how="left"           # keep all rows from df_extreme_risk_data
)

In [63]:
df_all_extreme.head()

,PERSON ID,CURRENT AGE_x,COMMITMENT COUNTY_x,RESPONSIBLE COUNTY_x,GENDER_x,REMOVAL ZIP CODE_x,PLACEMENT ZIP CODE_x,LOCATION BEGIN DATE_x,LOCATION END DATE_x,Decision-Making (Judgement),...,Mental_Health_Sexual_Developmental_Concerns,Mental_Health_Trauma_Stress_Related,Supportive_Adult_Relationships_Family_Relationships,Supportive_Adult_Relationships_Social_Relationships,Supportive_Adult_Relationships_Barriers_to_Support,Housing_Instability_Skills_Readiness,Housing_Instability_Current_Living_Situation,Housing_Instability_Barriers_Risks,County_y,Region_y
0,353258,17,Madison,Madison,MALE,37040.0,37040.0,2023-06-13,2024-05-25,0,...,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,Madison,West
1,706512,17,Sullivan,Sullivan,FEMALE,37642.0,37642.0,2024-11-18,NaN,0,...,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,Sullivan,East
2,1348374,17,Warren,Warren,FEMALE,37110.0,37110.0,2023-06-02,2023-09-30,0,...,0.0,NaN,NaN,1.0,NaN,NaN,0.0,0.0,Warren,Middle
3,2470086,18,Hamilton,Hamilton,MALE,37218.0,37218.0,2024-09-17,NaN,0,...,0.0,0.0,NaN,1.0,NaN,NaN,NaN,NaN,Hamilton,East
4,3965628,21,Davidson,Davidson,MALE,37725.0,37725.0,2021-12-22,2022-08-31,0,...,0.0,NaN,0.0,NaN,1.0,0.0,0.0,0.0,Davidson,Middle


In [64]:
df_all_extreme.columns.tolist()

['PERSON ID',
 'CURRENT AGE_x',
 'COMMITMENT COUNTY_x',
 'RESPONSIBLE COUNTY_x',
 'GENDER_x',
 'REMOVAL ZIP CODE_x',
 'PLACEMENT ZIP CODE_x',
 'LOCATION BEGIN DATE_x',
 'LOCATION END DATE_x',
 'Decision-Making (Judgement)',
 'Impulsivity/Hyperactivity',
 'School Behavior',
 'Substance Use',
 'Spiritual/Religious',
 'Traditions And Rituals',
 'Youth Involvement With Care',
 'Anger Control',
 'Anxiety',
 'Depression',
 'Emotional And/Or Physical Dysregulation',
 'Numbing',
 'Medical/Physical',
 'Psychosis (Thought Disorder)',
 'Danger To Others',
 'Intentional Misbehavior',
 'Non-Suicidal Self-Injurious Behavior',
 'Other Self-Harm (Recklessness)',
 'Suicide Risk',
 'Sexual Aggression',
 'Sexual Development',
 'Adjustment To Trauma',
 'Cultural Stress',
 'Emotional Abuse',
 'Environmental Influences',
 'Medical Trauma',
 'Sexual Abuse',
 'Attachment Difficulties',
 'Community Life',
 'Family Functioning',
 'Family Strengths',
 'Interpersonal/Social Connectedness',
 'Natural Supports',
 '

In [66]:
df_all_extreme.drop(columns=['CURRENT AGE_x',
 'COMMITMENT COUNTY_x',
 'RESPONSIBLE COUNTY_x',
 'GENDER_x',
 'REMOVAL ZIP CODE_x',
 'PLACEMENT ZIP CODE_x',
 'LOCATION BEGIN DATE_x',
 'LOCATION END DATE_x',
 'County_x',
 'Region_x'])

,PERSON ID,Decision-Making (Judgement),Impulsivity/Hyperactivity,School Behavior,Substance Use,Spiritual/Religious,Traditions And Rituals,Youth Involvement With Care,Anger Control,Anxiety,...,Mental_Health_Sexual_Developmental_Concerns,Mental_Health_Trauma_Stress_Related,Supportive_Adult_Relationships_Family_Relationships,Supportive_Adult_Relationships_Social_Relationships,Supportive_Adult_Relationships_Barriers_to_Support,Housing_Instability_Skills_Readiness,Housing_Instability_Current_Living_Situation,Housing_Instability_Barriers_Risks,County_y,Region_y
0,353258,0,0,0,0,0,0,0.0,0,0,...,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,Madison,West
1,706512,0,0,0,0,0,0,0.0,0,0,...,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,Sullivan,East
2,1348374,0,0,0,0,0,0,0.0,0,0,...,0.0,NaN,NaN,1.0,NaN,NaN,0.0,0.0,Warren,Middle
3,2470086,0,0,0,0,1,0,0.0,0,0,...,0.0,0.0,NaN,1.0,NaN,NaN,NaN,NaN,Hamilton,East
4,3965628,0,0,0,0,0,0,0.0,0,0,...,0.0,NaN,0.0,NaN,1.0,0.0,0.0,0.0,Davidson,Middle
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5662,135541144,0,0,0,0,1,0,0.0,0,0,...,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,Warren,Middle
5663,135654245,0,0,0,0,1,0,1.0,0,0,...,NaN,1.0,1.0,1.0,1.0,NaN,1.0,0.0,Hamilton,East
5664,135694181,0,0,0,0,1,0,0.0,0,0,...,0.0,0.0,NaN,1.0,NaN,0.0,0.0,0.0,Wilson,Middle
5665,135731227,0,0,0,0,1,0,0.0,0,0,...,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,Sumner,Middle


In [70]:
# Rename columns to remove '_x' suffix
df_all_extreme = df_all_extreme.rename(
    columns={col: col.replace('_y', '') for col in df_all_extreme.columns if col.endswith('_y')}
)


In [71]:
# Percent of ones per county for each flag column
pct_ones_by_county_extreme = (
    df_all_extreme.groupby('County')[calc_cols2]
    .mean()
    .mul(100)
    .round(2)
    .reset_index()
)
pct_ones_by_county_extreme

,County,Decision-Making (Judgement),Impulsivity/Hyperactivity,School Behavior,Substance Use,Spiritual/Religious,Traditions And Rituals,Youth Involvement With Care,Anger Control,Anxiety,...,Mental_Health_Psychotic_Thought_Disturbances,Mental_Health_Risk_Safety,Mental_Health_Sexual_Developmental_Concerns,Mental_Health_Trauma_Stress_Related,Supportive_Adult_Relationships_Family_Relationships,Supportive_Adult_Relationships_Social_Relationships,Supportive_Adult_Relationships_Barriers_to_Support,Housing_Instability_Skills_Readiness,Housing_Instability_Current_Living_Situation,Housing_Instability_Barriers_Risks
0,Anderson,1.83,0.92,2.75,0.92,46.79,0.92,5.62,2.75,0.00,...,0.0,11.54,2.67,70.59,90.00,98.46,81.82,8.70,14.71,26.47
1,Bedford,2.00,0.00,0.00,0.00,44.00,0.00,0.00,2.00,0.00,...,0.0,0.00,2.78,30.00,71.43,97.06,55.56,17.65,15.38,11.11
2,Benton,23.08,0.00,0.00,0.00,30.77,0.00,0.00,0.00,0.00,...,0.0,NaN,0.00,100.00,100.00,100.00,NaN,0.00,60.00,0.00
3,Bledsoe,0.00,0.00,0.00,0.00,33.33,0.00,0.00,6.67,0.00,...,0.0,0.00,0.00,20.00,0.00,77.78,33.33,0.00,25.00,0.00
4,Blount,2.22,1.11,2.22,0.00,48.89,0.00,0.00,1.11,0.00,...,0.0,33.33,4.26,88.24,100.00,97.78,88.24,3.70,45.83,24.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,Wayne,3.23,3.23,3.23,3.23,25.81,0.00,6.67,3.23,0.00,...,0.0,16.67,9.52,100.00,100.00,100.00,100.00,0.00,12.50,12.50
91,Weakley,5.88,0.00,0.00,5.88,41.18,0.00,0.00,0.00,0.00,...,0.0,0.00,0.00,50.00,100.00,100.00,100.00,10.00,50.00,25.00
92,White,2.63,2.63,0.00,0.00,39.47,0.00,3.03,0.00,0.00,...,0.0,11.11,4.00,0.00,100.00,100.00,100.00,0.00,21.43,0.00
93,Williamson,9.23,4.62,1.54,4.62,46.15,0.00,2.33,6.15,1.54,...,0.0,20.00,3.92,16.67,87.50,98.08,44.44,0.00,18.18,25.81


In [72]:
pct_ones_by_county_extreme.to_csv('pct_ones_extreme_risk_by_county.csv', index=False)